In [ ]:
# Load dataset in Pytorch dataset class

from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

trainset = datasets.CIFAR10("content", train=True,  download=True)
testset = datasets.CIFAR10("content", train = False, download=True)

# Display some samples
for j in range(10):
  image, class_num = trainset[j]
  plt.subplot(1, 10 ,j+1)
  plt.imshow(image)
  plt.axis('off')
  plt.title(str(class_num))

In [ ]:
# Validation function. Don't cahnge it
def validate(model,testloader):
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
  
  return correct / total

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F


class TwoLayerFCN(nn.Module):

    size = 32 * 32 * 3

    def __init__(self, class_nums = 10):
        super(TwoLayerFCN, self).__init__()
        # Define two layer fully - connected network with linear layers: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear

        # Put your code here
        self.layer1 = nn.Linear(self.size, 256)
        self.layer2 = nn.Linear(256, class_nums)
    

    def forward(self, x):
        # Preprocess image, remember that first dimension is batch dimension !

        # Perform forward pass;
        # Use layers defined in constructor;
        # Add activation function on your choice: https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
        
        # Put your code here
        x = x.view(-1, self.size)
        scores = F.relu(self.layer1(x))
        scores = self.layer2(scores)

        # return raw output of last layer
        return scores


In [ ]:
# Prepare the data
transform=transforms.Compose([
                              transforms.ToTensor(), # Pil Image to Pytorch tensor
                              transforms.Normalize((0.491, 0.482, 0.446), (0.247, 0.243, 0.261)) # https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.Normalize
                              ])

trainset.transform = transform
testset.transform = transform

train_loader = DataLoader(trainset, batch_size = 512, shuffle = True)
test_loader = DataLoader(testset, batch_size = 256, shuffle = False)



In [ ]:
# Train the model
# You may change loss function, learning rate and number of epochs

model = TwoLayerFCN(10) 
model.train()

# By default cross-entropy loss used, you can change it to another loss function
criterion = nn.CrossEntropyLoss() 

# You can change learning rate and number of epoch too
optimizer = torch.optim.SGD(model.parameters(), lr=0.005) 
for epoch in range(25):
    for img_batch, labels_batch in train_loader:

        output = model(img_batch)
        loss = criterion(output, labels_batch)
        # Get the grad and update model weights: https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#backprop

        # Put your code here
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    accuracy = validate(model,test_loader)
    print("Epoch {} Loss {:.2f} Accuracy {:.2f}".format(epoch,loss.item(),accuracy))
  

Epoch 0 Loss 1.99 Accuracy 0.31
Epoch 1 Loss 1.86 Accuracy 0.35
Epoch 2 Loss 1.88 Accuracy 0.37
Epoch 3 Loss 1.78 Accuracy 0.39
Epoch 4 Loss 1.70 Accuracy 0.40
Epoch 5 Loss 1.75 Accuracy 0.41
Epoch 6 Loss 1.70 Accuracy 0.41
Epoch 7 Loss 1.77 Accuracy 0.42
Epoch 8 Loss 1.65 Accuracy 0.42
Epoch 9 Loss 1.66 Accuracy 0.43
Epoch 10 Loss 1.60 Accuracy 0.44
Epoch 11 Loss 1.55 Accuracy 0.44
Epoch 12 Loss 1.65 Accuracy 0.44
Epoch 13 Loss 1.59 Accuracy 0.45
Epoch 14 Loss 1.62 Accuracy 0.45
Epoch 15 Loss 1.54 Accuracy 0.45
Epoch 16 Loss 1.61 Accuracy 0.46
Epoch 17 Loss 1.52 Accuracy 0.46
Epoch 18 Loss 1.51 Accuracy 0.46
Epoch 19 Loss 1.44 Accuracy 0.46
Epoch 20 Loss 1.61 Accuracy 0.46
Epoch 21 Loss 1.58 Accuracy 0.47
Epoch 22 Loss 1.54 Accuracy 0.47
Epoch 23 Loss 1.48 Accuracy 0.47
Epoch 24 Loss 1.49 Accuracy 0.47


## Ideas for extra work

* Compare different optimizators 
* Compare different Activation functions
* Evaluate Batch Normalization effect (need a deeper network) 